## Setting up notebook

In [1]:
# installing packages
!pip install pip==24.0
!pip install numpy==1.23.5
!pip install tensorboardX
!pip install subword-nmt
!pip install sentencepiece

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with 

In [2]:
# importing packages
import numpy
import os
import tensorboardX
import sentencepiece as spm

In [3]:
# mounting google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
source_code = 'eng'
target_code = 'tso'

In [5]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}')

# installing fairseq
#!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
!pip install --editable ./

/content/drive/MyDrive/Research/eng-to-tso/fairseq
Obtaining file:///content/drive/MyDrive/Research/eng-to-tso/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for fairseq (pyproject.toml) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-0.editable-cp310-cp310-linux_x86_64.whl size=9580 sha256=df8833bac01126d9c59351538cfd5df0a17d123926e91b41b85fa86088ce1c82
  Stored in directory: /tmp/pip-ephem-wheel-cache-dj27xpld/wheels/9e/98/8e/ef4c5761efd94805bbd4d6c28ef61a86b874a20d6696dba7ca
Successfully built fairseq
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency

## Tokenizing the data

In [6]:
train_source_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/train.{source_code}'
train_target_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/train.{target_code}'
val_source_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/val.{source_code}'
val_target_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/val.{target_code}'
test_source_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/test.{source_code}'
test_target_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/test.{target_code}'

### BPE

In [7]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpe/data')

In [8]:
# learn joint vocabulary
!subword-nmt learn-joint-bpe-and-vocab --input $train_source_path $train_target_path -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$source_code vocab.$target_code

100% 4000/4000 [00:08<00:00, 447.54it/s] 


In [9]:
# applying tokenization to training, validation and test sets
!subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$source_code < $train_source_path > train.BPE.$source_code
!subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$target_code < $train_target_path > train.BPE.$target_code

!subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$source_code < $val_source_path > val.BPE.$source_code
!subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$target_code < $val_target_path > val.BPE.$target_code

!subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$source_code < $test_source_path > test.BPE.$source_code
!subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$target_code < $test_target_path > test.BPE.$target_code

In [10]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpe')

In [11]:
# prepare dictionary to be used by our nmt model
!fairseq-preprocess -s eng -t tso \
--trainpref data/train.BPE \
--validpref data/val.BPE \
--testpref data/test.BPE \
--joined-dictionary \
--destdir data-bin \
--bpe 'subword_nmt' \
--thresholdsrc 2 \
--thresholdtgt 2 \
--workers 2

2024-10-15 11:50:55.513832: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 11:50:55.863881: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 11:50:55.960462: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 11:50:56.536378: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-15 11:50:58.323375: W tensorflow/comp

### ULM

In [12]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulm/data')

In [13]:
# learn joint vocabulary
spm.SentencePieceTrainer.train(f'--input={train_source_path},{train_target_path} --model_prefix=joint --model_type=unigram --vocab_size=4000 --character_coverage=1.0 --bos_id=0 --pad_id=1 --eos_id=2 --unk_id=3')

In [14]:
# applying tokenization to training, validation and test sets
sp = spm.SentencePieceProcessor(model_file='joint.model')

with open(train_source_path, 'r') as rf, open(f'train.ULM.{source_code}', 'w') as wf:
    for line in rf:
        wf.write(' '.join(sp.encode(line, out_type=str)) + '\n')

with open(train_target_path, 'r') as rf, open(f'train.ULM.{target_code}', 'w') as wf:
    for line in rf:
        wf.write(' '.join(sp.encode(line, out_type=str)) + '\n')

with open(val_source_path, 'r') as rf, open(f'val.ULM.{source_code}', 'w') as wf:
    for line in rf:
        wf.write(' '.join(sp.encode(line, out_type=str)) + '\n')

with open(val_target_path, 'r') as rf, open(f'val.ULM.{target_code}', 'w') as wf:
    for line in rf:
        wf.write(' '.join(sp.encode(line, out_type=str)) + '\n')

with open(test_source_path, 'r') as rf, open(f'test.ULM.{source_code}', 'w') as wf:
    for line in rf:
        wf.write(' '.join(sp.encode(line, out_type=str)) + '\n')

with open(test_target_path, 'r') as rf, open(f'test.ULM.{target_code}', 'w') as wf:
    for line in rf:
        wf.write(' '.join(sp.encode(line, out_type=str)) + '\n')

In [15]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulm')

In [16]:
# prepare dictionary to be used by our nmt model
!fairseq-preprocess -s eng -t tso \
--trainpref data/train.ULM \
--validpref data/val.ULM \
--testpref data/test.ULM \
--joined-dictionary \
--destdir data-bin \
--bpe 'sentencepiece' \
--thresholdsrc 2 \
--thresholdtgt 2 \
--workers 2

2024-10-15 11:52:49.406189: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 11:52:49.434403: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 11:52:49.440793: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 11:52:49.456437: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-15 11:52:50.558873: W tensorflow/comp